In [1]:
import logging

from cicada.communicator import SocketCommunicator
from cicada.logging import Logger

logging.basicConfig(level=logging.INFO)

def main(parent):
    log = Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name} ranks: {parent.ranks}")
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Player 0 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 1 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: world ranks: [0, 1, 2, 3]


In [2]:
def main(parent):
    log = Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name} ranks: {parent.ranks}")
    with parent.split(name="child") as child:
        log.info(f"Player {parent.rank} child name: {child.name} ranks: {child.ranks}")
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Player 0 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 1 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 0 child name: child ranks: [0, 1, 2, 3]
INFO:root:Player 1 child name: child ranks: [0, 1, 2, 3]
INFO:root:Player 2 child name: child ranks: [0, 1, 2, 3]
INFO:root:Player 3 child name: child ranks: [0, 1, 2, 3]


In [3]:
def main(parent):
    log = Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name} ranks: {parent.ranks}")
    
    if parent.rank in [0, 1]:
        name = "red"
    if parent.rank in [2, 3]:
        name = "blue"
    
    with parent.split(name=name) as child:
        log.info(f"Player {parent.rank} child name: {child.name} ranks: {child.ranks}")
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Player 0 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 1 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: world ranks: [0, 1, 2, 3]
INFO:root:Player 0 child name: red ranks: [0, 1]
INFO:root:Player 1 child name: red ranks: [0, 1]
INFO:root:Player 2 child name: blue ranks: [0, 1]
INFO:root:Player 3 child name: blue ranks: [0, 1]


In [4]:
import collections
import logging

import numpy

Game = collections.namedtuple("Game", ["communicator", "log"])

def main(communicator):
    # Setup multiple games with separate communicators.
    games = []
    partitions = [[0, 1, 2], [1, 2, 3], [2, 3, 0]]
    for index, partition in enumerate(partitions):
        game_communicator = communicator.split(name=f"game-{index}" if communicator.rank in partition else None)
        if game_communicator is not None:
            game = Game(
                communicator=game_communicator,
                log=Logger(logging.getLogger(), game_communicator),
                )
            games.append(game)

    # Run games in round-robin fashion.
    for i in range(2):
        for game in games:
            value = f"{game.communicator.name} message {i}" if game.communicator.rank == 0 else None
            value = game.communicator.broadcast(src=0, value=value)
            game.log.info(f"{game.communicator.name} player {game.communicator.rank} received broadcast value: {value}")

    # Cleanup games.
    for game in games:
        game.communicator.free()

SocketCommunicator.run(world_size=4, fn=main);

INFO:root:game-0 player 0 received broadcast value: game-0 message 0
INFO:root:game-0 player 1 received broadcast value: game-0 message 0
INFO:root:game-0 player 2 received broadcast value: game-0 message 0
INFO:root:game-1 player 0 received broadcast value: game-1 message 0
INFO:root:game-2 player 0 received broadcast value: game-2 message 0
INFO:root:game-1 player 1 received broadcast value: game-1 message 0
INFO:root:game-1 player 2 received broadcast value: game-1 message 0
INFO:root:game-2 player 1 received broadcast value: game-2 message 0
INFO:root:game-2 player 2 received broadcast value: game-2 message 0
INFO:root:game-0 player 0 received broadcast value: game-0 message 1
INFO:root:game-0 player 1 received broadcast value: game-0 message 1
INFO:root:game-0 player 2 received broadcast value: game-0 message 1
INFO:root:game-2 player 0 received broadcast value: game-2 message 1
INFO:root:game-1 player 0 received broadcast value: game-1 message 1
INFO:root:game-1 player 1 received